<a href="https://colab.research.google.com/github/dakyommii/study/blob/main/test/segmentation_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#segmentation-train

###train

In [ ]:
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.models import Model
from keras.layers import Input, concatenate, Dropout, Reshape, Permute, Activation, ZeroPadding2D, Cropping2D, Add
from keras.layers.convolutional import Conv2D, MaxPooling2D, UpSampling2D, Conv2DTranspose, DepthwiseConv2D
from keras.regularizers import l2
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

In [ ]:
IMG_HEIGHT = 256
IMG_WIDTH = 256
BATCH_SIZE = 16

x_train = np.load('/content/drive/MyDrive/segmentation/data/x_train.npz')['data'].astype(np.float32)
y_train = np.load('/content/drive/MyDrive/segmentation/data/y_train.npz')['data'].astype(np.float32)
x_val = np.load('/content/drive/MyDrive/segmentation/data/x_val.npz')['data'].astype(np.float32)
y_val = np.load('/content/drive/MyDrive/segmentation/data/y_val.npz')['data'].astype(np.float32)

print(x_train.shape, y_train.shape)
print(x_val.shape, y_val.shape)

(4000, 256, 256, 3) (4000, 256, 256, 2)
(80, 256, 256, 3) (80, 256, 256, 2)


In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    brightness_range=[0.7, 1.3]
)

val_datagen = ImageDataGenerator(
    rescale=1./255
)

train_gen = train_datagen.flow(
    x_train,
    y_train,
    batch_size=BATCH_SIZE,
    shuffle=True
)

val_gen = val_datagen.flow(
    x_val,
    y_val,
    batch_size=BATCH_SIZE,
    shuffle=False
)

In [ ]:
inputs = Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3))

# encode
conv1 = Conv2D(32, 3, activation='relu', padding='same', kernel_initializer='orthogonal')(inputs)
# conv1 = Dropout(0.2)(conv1)
conv1 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='orthogonal')(conv1)
pool1 = MaxPooling2D(pool_size=2)(conv1)
# (128, 128, 64)

shortcut_1 = pool1

conv2 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='orthogonal')(pool1)
# conv2 = Dropout(0.2)(conv2)
conv2 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='orthogonal')(conv2)
pool2 = MaxPooling2D(pool_size=2)(conv2)
# (64, 64, 128)

shortcut_2 = pool2

conv3 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='orthogonal')(pool2)
# conv3 = Dropout(0.2)(conv3)
conv3 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='orthogonal')(conv3)
pool3 = MaxPooling2D(pool_size=2)(conv3)
# (32, 32, 256)

# middle depthwiseconv nodes
shortcut_3 = pool3

mid = DepthwiseConv2D(3, activation='relu', padding='same', kernel_initializer='orthogonal')(pool3)
mid = Conv2D(256, 1, activation='relu', padding='same', kernel_initializer='orthogonal')(mid)

mid = DepthwiseConv2D(3, activation='relu', padding='same', kernel_initializer='orthogonal')(mid)
mid = Conv2D(256, 1, activation='relu', padding='same', kernel_initializer='orthogonal')(mid)

mid = Add()([shortcut_3, mid])

# decode
up8 = UpSampling2D(size=2)(mid)
up8 = concatenate([up8, conv3], axis=-1)
conv8 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='orthogonal')(up8)
# conv8 = Dropout(0.2)(conv8)
conv8 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='orthogonal')(conv8)
# (64, 64, 128)

conv8 = Add()([shortcut_2, conv8])

up9 = UpSampling2D(size=2)(conv8)
up9 = concatenate([up9, conv2], axis=-1)
conv9 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='orthogonal')(up9)
# conv9 = Dropout(0.2)(conv9)
conv9 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='orthogonal')(conv9)
# (128, 128, 64)

conv9 = Add()([shortcut_1, conv9])

up10 = UpSampling2D(size=2)(conv9)
up10 = concatenate([up10, conv1], axis=-1)
conv10 = Conv2D(32, 3, activation='relu', padding='same', kernel_initializer='orthogonal')(up10)
# conv10 = Dropout(0.2)(conv10)
conv10 = Conv2D(32, 3, activation='relu', padding='same', kernel_initializer='orthogonal')(conv10)
# (256, 256, 32)

conv11 = Conv2D(2, 1, padding='same', activation='relu',
                kernel_initializer='he_normal', kernel_regularizer=l2(0.005))(conv10)
conv11 = Reshape((IMG_HEIGHT * IMG_WIDTH, 2))(conv11)
# (256, 256, 2)

conv11 = Activation('softmax')(conv11)

outputs = Reshape((IMG_HEIGHT, IMG_WIDTH, 2))(conv11)

model = Model(inputs=inputs, outputs=outputs)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 256, 256, 32  896         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 conv2d_1 (Conv2D)              (None, 256, 256, 64  18496       ['conv2d[0][0]']                 
                                )                                                             

In [ ]:
history = model.fit_generator(train_gen,
    validation_data=val_gen,
    epochs=100,
    callbacks=[
        ModelCheckpoint('/content/drive/MyDrive/segmentation/models/unet_no_drop.h5', monitor='val_acc', save_weights_only=True, verbose=1),
        ReduceLROnPlateau(monitor='val_acc', factor=0.2, patience=10, verbose=1, min_lr=1e-05)

    ]
)

Epoch 1/100


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


250/250 [==============================] - ETA: 0s - loss: 0.0505 - accuracy: 0.9937
Epoch 1: saving model to /content/drive/MyDrive/segmentation/models/unet_no_drop.h5
250/250 [==============================] - 110s 373ms/step - loss: 0.0505 - accuracy: 0.9937 - val_loss: 0.0071 - val_accuracy: 0.9998 - lr: 0.0010
Epoch 2/100
250/250 [==============================] - ETA: 0s - loss: 0.0403 - accuracy: 0.9976
Epoch 2: saving model to /content/drive/MyDrive/segmentation/models/unet_no_drop.h5
250/250 [==============================] - 92s 369ms/step - loss: 0.0403 - accuracy: 0.9976 - val_loss: 0.0045 - val_accuracy: 0.9998 - lr: 0.0010
Epoch 3/100
250/250 [==============================] - ETA: 0s - loss: 0.0392 - accuracy: 0.9976
Epoch 3: saving model to /content/drive/MyDrive/segmentation/models/unet_no_drop.h5
250/250 [==============================] - 93s 370ms/step - loss: 0.0392 - accuracy: 0.9976 - val_loss: 0.0041 - val_accuracy: 0.9998 - lr: 0.0010
Epoch 4/100
250/250 [======

###test

In [ ]:
from keras.models import load_model
import matplotlib.pyplot as plt
import cv2
import numpy as np

import os

In [ ]:
model = load_model('models/unet_no_drop.h5')

In [ ]:
IMG_PATH = 'imgs/04.jpg'

img = cv2.imread(IMG_PATH, cv2.IMREAD_COLOR)
img_ori = cv2.cvtColor(img.copy(), cv2.COLOR_BGR2RGB)

plt.figure(figsize=(16, 16))
plt.imshow(img_ori)

In [ ]:
IMG_WIDTH, IMG_HEIGHT = 256, 256

def preprocess(img):
    im = np.zeros((IMG_WIDTH, IMG_HEIGHT, 3), dtype=np.uint8)

    if img.shape[0] >= img.shape[1]:
        scale = img.shape[0] / IMG_HEIGHT
        new_width = int(img.shape[1] / scale)
        diff = (IMG_WIDTH - new_width) // 2
        img = cv2.resize(img, (new_width, IMG_HEIGHT))

        im[:, diff:diff + new_width, :] = img
    else:
        scale = img.shape[1] / IMG_WIDTH
        new_height = int(img.shape[0] / scale)
        diff = (IMG_HEIGHT - new_height) // 2
        img = cv2.resize(img, (IMG_WIDTH, new_height))

        im[diff:diff + new_height, :, :] = img
        
    return im

img = preprocess(img)

plt.figure(figsize=(8, 8))
plt.imshow(img)

In [ ]:
input_img = img.reshape((1, IMG_WIDTH, IMG_HEIGHT, 3)).astype(np.float32) / 255.

pred = model.predict(input_img)

In [ ]:
THRESHOLD = 0.5
EROSION = 1

def postprocess(img_ori, pred):
    h, w = img_ori.shape[:2]
    
    mask_ori = (pred.squeeze()[:, :, 1] > THRESHOLD).astype(np.uint8)
    max_size = max(h, w)
    result_mask = cv2.resize(mask_ori, dsize=(max_size, max_size))

    if h >= w:
        diff = (max_size - w) // 2
        if diff > 0:
            result_mask = result_mask[:, diff:-diff]
    else:
        diff = (max_size - h) // 2
        if diff > 0:
            result_mask = result_mask[diff:-diff, :]
        
    result_mask = cv2.resize(result_mask, dsize=(w, h))
    
    # fill holes
#     cv2.floodFill(result_mask, mask=np.zeros((h+2, w+2), np.uint8), seedPoint=(0, 0), newVal=255)
#     result_mask = cv2.bitwise_not(result_mask)
    result_mask *= 255

#     # erode image
#     element = cv2.getStructuringElement(cv2.MORPH_RECT, (2*EROSION + 1, 2*EROSION+1), (EROSION, EROSION))
#     result_mask = cv2.erode(result_mask, element)

    # smoothen edges
    result_mask = cv2.GaussianBlur(result_mask, ksize=(9, 9), sigmaX=5, sigmaY=5)
    
    return result_mask

mask = postprocess(img_ori, pred)

plt.figure(figsize=(16, 16))
plt.subplot(1, 2, 1)
plt.imshow(pred[0, :, :, 1])
plt.subplot(1, 2, 2)
plt.imshow(mask)

In [ ]:
converted_mask = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)

result_img = cv2.subtract(converted_mask, img_ori)
result_img = cv2.subtract(converted_mask, result_img)

plt.figure(figsize=(16, 16))
plt.imshow(result_img)

In [ ]:
bg_img = cv2.imread('imgs/monalisa.jpg')
bg_img = cv2.cvtColor(bg_img, cv2.COLOR_BGR2RGB)

print(bg_img.shape, result_img.shape)

plt.figure(figsize=(16, 16))
plt.imshow(bg_img)